# Using R with pyABC

In [ ]:
%matplotlib inline

In [ ]:
from pyabc.external import R

r = R("myRModel.R")
r.display_source_ipython()

In [ ]:
model = r.model("myModel")
distance = r.distance("myDistance")
sum_stat = r.summary_statistics("mySummaryStatistics")

In [ ]:
from pyabc import Distribution, RV, ABCSMC

prior = Distribution(meanX=RV("uniform", 0, 10),
                     meanY=RV("uniform", 0, 10))
abc = ABCSMC(model, prior, distance,
             summary_statistics=sum_stat)

In [ ]:
import os
from tempfile import gettempdir

db = "sqlite:///" + os.path.join(gettempdir(), "test.db")
abc.new(db, r.observation("myObservation"))

In [ ]:
history = abc.run(minimum_epsilon=0.3, max_nr_populations=8)

In [ ]:
from pyabc.visualization import plot_hist_2d

df, w = abc.history.get_distribution(0)
ax = plot_hist_2d(df, w, "meanX", "meanY",
                  xmin=0, xmax=10,
                  ymin=0, ymax=10,
                  numx=200, numy=200)
ax.scatter([4], [8], edgecolor="black", facecolor="white", label="Ground Truth");
ax.legend();